In [1]:
import pandas as pd
import numpy as np

In [2]:
train = pd.read_csv("train.csv")
# 데이터 읽어드리기

In [3]:
train.head()
# 데이터의 형태를 참고해본다.

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [4]:
train.loc[train[np.isnan(train['Age'])].index, "Age"] = round(train["Age"].mean(), 0)
train["Embarked"][train["Embarked"].isnull()] = "S"
# Age에 대한 결측치는 전체 나이의 평균을 대입했고
# Embarked에 대한 결측치는 S로 처리했다.

C:\Users\ewon\Anaconda3\envs\ml_practice\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [5]:
del train[train.columns[0]] # passenger_id 처리
del train["Ticket"]
del train["Name"]
del train["Cabin"]
train["familySize"] = train["SibSp"] + train["Parch"]
del train["SibSp"]
del train["Parch"]
train["travelAlone"] = np.where(train["familySize"]>0, 0,1)
train.loc[train.loc[train['Sex']=='male'].index, "Sex"] = 0
train.loc[train.loc[train['Sex']=='female'].index, "Sex"] = 1
train.loc[train.loc[train['Embarked']=='S'].index, "Embarked"] = 0
train.loc[train.loc[train['Embarked']=='C'].index, "Embarked"] = 1
train.loc[train.loc[train['Embarked']=='Q'].index, "Embarked"] = 2

# Ticket, Name, Cabin 은 불필요한 데이터로 하여 제거하였다.
# SibSp 와 Parch를 이용해 familySize를 만들고
# familySize 가 0일 경우 travelAlone 라는 변수를 추가하여 1 을 대입했다.
# Sex, Embarked에 대한 데이터를 학습시키기 편하게 전처리하였다.

In [6]:
train

,Survived,Pclass,Sex,Age,Fare,Embarked,familySize,travelAlone
0,0,3,0,22.0,7.2500,0,1,0
1,1,1,1,38.0,71.2833,1,1,0
2,1,3,1,26.0,7.9250,0,0,1
3,1,1,1,35.0,53.1000,0,1,0
4,0,3,0,35.0,8.0500,0,0,1
5,0,3,0,30.0,8.4583,2,0,1
6,0,1,0,54.0,51.8625,0,0,1
7,0,3,0,2.0,21.0750,0,4,0
8,1,3,1,27.0,11.1333,0,2,0
9,1,2,1,14.0,30.0708,1,1,0


In [7]:
target = train.loc[:,"Survived"]
target =  np.asarray(target, dtype = "int")
values = train.drop(["Survived"], axis = 1)

# Survived를 target 변수로 뽑아낸다.

In [8]:
values.isnull().sum()

Pclass         0
Sex            0
Age            0
Fare           0
Embarked       0
familySize     0
travelAlone    0
dtype: int64

In [9]:
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score

In [10]:
kf = KFold(n_splits=10)
gn = GaussianNB()

In [29]:
list_ = []

In [31]:
# fit에는 독립변수들은 dataframe 형태로 
# 종속변수는 numpy.ndarray 형태로 들어간다.
for train_index, test_index in kf.split(values):
    target_train, target_test = target[train_index], target[test_index]
    values_train, values_test = values.iloc[train_index], values.iloc[test_index]
    nb_model = gn.fit(values_train, target_train)
    list_ = np.append(list_,cross_val_score(nb_model, values_test, target_test).mean())

In [34]:
round(list_.mean()*100,3)
# 총평균

75.937